### **1.) Import all of these packages**

In [ ]:
import numpy as np
import struct
import array
import pandas as pd

import ipywidgets as widgets
import matplotlib.pyplot as plt
import re
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator, MaxNLocator)

from lmfit import Model
from lmfit.model import save_modelresult, load_modelresult

import sys
import tkinter
from tkinter.filedialog import askopenfilenames

### **2.) Run this block to initialize all the functions**

In [ ]:
def get_file_names():    
    root = tkinter.Tk()
    root.withdraw()
    root.attributes('-topmost',True)
    file_list = askopenfilenames(parent=root, title='Open CSV file',
                                              filetypes=[('.csv', '*.csv')])
    file_list = root.tk.splitlist(file_list) #workaround for Windows bug
    root.destroy()
    return file_list

def get_import_fit():    
    root = tkinter.Tk()
    root.withdraw()
    root.attributes('-topmost',True)
    file_list = askopenfilenames(parent=root, title='Import XPS fit',
                                              filetypes=[('.JSON', '*.JSON')])
    file_list = root.tk.splitlist(file_list) #workaround for Windows bug
    root.destroy()
    return file_list

### Shirley background algorithm

### Brief explanation of the algorithm: Shirley background is generated to be S(E) = y_min + interval * { A(2) / [A(1)+A(2)] }
##  Where S(E) is the background intensity at a given binding energy E
##  A(2) is the area at binding energies less than E, and A(1) is area at binding energies greater than E
##  interval is usually accepted to be the intensity difference of the lower and upper bounds for binding energy, for our background region of interest
##  This implementation first calculates the total area [A(1) + A(2)] by numerically integrating via the trapezoid method to find k_integral, and then k_val is simply " interval / [A(1) + A(2)]"
##  Next we calculate A(2) (designated y_integral) and multiply it with k_val
##  Once the iteration has completed, we add y_min to the background
##  Tolerance is determined by calculating the norm of the difference of the initial and final background arrays. I don't understand how that works, but norm basically indicates the degree of difference between 2 arrays, so the smaller the norm, the higher tolerance. Unit is arbitrary, I think.
def shirley_func(ind_min,ind_max,be_vals,intensity_vals):
    shirley_back = np.zeros(be_vals.shape)      ## initialize shirley_background array, same size as the entire domain of the region
    for i in range(ind_max,ind_min+1):          ## populate shirley_back array with values for linear background
        shirley_back[i] = ((y_max - y_min)/(x_max - x_min)) * (be_vals[i] - x_min)
    shirley_back_it = shirley_back.copy()       ## copy shirley_back array for a separate, iterated/manipulated version

    tol = 1e-12      ## Tolerance level

    it_max = 10       ## maximum number of iterations
    it = 0              ## initial iteration value

    while it < it_max:
        k_integral = 0.0
        for i in range(ind_min, ind_max):
            k_integral += (be_vals[i+1] - be_vals[i]) * 0.5 * (intensity_vals[i+1] + intensity_vals[i] - 2 * y_min - shirley_back[i+1] - shirley_back[i]) # integration via trapezoid method
        k_val = (y_max - y_min)/k_integral
            
        for i in range(ind_min, ind_max):
            y_integral = 0.0
            for j in range(i, ind_max):
                y_integral += (be_vals[j+1] - be_vals[j]) * 0.5 * (intensity_vals[j+1] + intensity_vals[j] - 2 * y_min - shirley_back[j+1] - shirley_back[j])
            shirley_back_it[i] = k_val * y_integral
        
        if np.linalg.norm(shirley_back_it - shirley_back) < tol:
            shirley_back = shirley_back_it.copy()
            break
        else:
            shirley_back = shirley_back_it.copy()
        it +=1

    if it >= it_max:
        print("max iterations exceeded")

    shirley_back_fin = y_min + shirley_back

    print("Number of iterations: " + str(it))
    return shirley_back_fin
    # print(shirley_back_fin)

    # print(shirley_back_fin)e(figsize=(10,5))

### Gaussian
def gauss(x,E,F,area,m):
    # E = pars['E']
    # F = pars['F']
    # # a = param[2]
    # m = pars['m']
    # model = height* np.exp(-4 * np.log(2) * (1 - m/100) * ((x - E) / F)**2)
    model = (2 * np.sqrt(np.log(2))/(F * np.sqrt(np.pi))) * area * np.exp(-4 * np.log(2) * (1 - m/100) * ((x - E) / F)**2)
    return model

# Note: for Gaussian, FWHM = 2 * sigma * sqrt(2 ln(2))

### Lorentzian
def lorentz(x,E,F,area,m):
    # E = pars['E']
    # F = pars['F']
    # # a = param[2]
    # m = pars['m']
    # model = height / ((1 + 4 * m/100 * ((x - E) / F)**2))
    model = 2 * area / (np.pi * (1 + 4 * m/100 * ((x - E) / F)**2) * F)
    return model
# !!!! I don't understand why this equation shouldn't be multiplied by 2. !!!!

# Note: for Lorentzian, FWHM = 2 sigma

### E = position (eV); F = FWHM; m = % Lorentzian

### Sum G/L
def sum_gl(x,E,F,area,m):
    # E = param[0]
    # F = param[1]
    # a = param[2]
    # m = param[3]
    model = ((1-m/100) * gauss(x,E,F,area,0) + (m/100) * lorentz(x,E,F,area,100))
    return model

### Product G/L
## Has issues with returning the correct area. I think it has to do with how the weight factor 'm' is incorporated. May need to do numerical integration.
def product_gl(x,E,F,area,m):
    # E = pars['E']
    # F = pars['F']
    # # a = param[2]
    # m = pars['m']
    model = (gauss(x,E,F,area,m) * lorentz(x,E,F,area,m))
    return model

### Defining default variable valus 
class xps_settings():
    def __init__(self, region="39amu", background_lower_bound=None, background_upper_bound=None, num_peaks=1, param=[250,10.0,50,80], peak_id=['p1_']):
        self.region = region
        self.background_lower_bound = background_lower_bound
        self.background_upper_bound = background_upper_bound
        self.num_peaks = num_peaks
        self.param = param
        self.peak_id = peak_id

### **3.) Import the CSV**

In [ ]:
### Import file via GUI prompt
# file = get_file_names()
# print(file[0])

### Import file by manually inputting the path
path = "C:/Users/hm115/Google Drive/Forschung/Data/TPD/Kevin/"
file = "2204152_130k_0pt5MLPt_mo2n_113to800k.csv"

# data = pd.read_csv(file[0],header=[53])
data = pd.read_csv(f"{path}{file}",header=[53])
# print(data)

### **4.) Create your configuration**

Here's how the configuration works:

peak\_input = xps\_settings( &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; This line initializes the peak configuration <br /><br />
&emsp;&emsp;&emsp;&emsp;   region =  <br />
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Region name of interest. Make sure name is in "quotes". Default = "39amu" <br /><br />
&emsp;&emsp;&emsp;&emsp;    background\_lower\_bound = <br />
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Pick the lower temperature for the Shirley Background calculation to start at. Default = lower bound of region <br /><br />
&emsp;&emsp;&emsp;&emsp;    background\_upper\_bound = <br />
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Pick the higher temperature for the Shirley Background calculation to start at. Default = upper bound of region <br /><br />
&emsp;&emsp;&emsp;&emsp;    num\_peaks = <br />
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Number of peaks you want to fit in your region. Default = 1 <br /><br />
&emsp;&emsp;&emsp;&emsp;    param = <br />
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; Peak parameters. Format is as follows: \[\[Temperature, FWHM, Peak Area, % Gaussian/Lorentzian], ... ]. Default = \[\[250,10.0,50,80]]<br />
&emsp;&emsp;&emsp;&emsp;    peak\_id = <br />
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; List of peak ids for distinguising each peak. Label them however you will, but number of peak ids must be the same as the number for num_peaks. Default = ['p1_'] <br /><br />
)

In [ ]:
## 39amu
peak_input = xps_settings(
    region = "39amu",
    background_lower_bound = 170,
    background_upper_bound = 500,
    num_peaks = 3,
    param = [[260,10.0,100,80],[300,10,30,80],[350,10.0,25,80]],
    peak_id = ['p1_','p2_','p3_'],
    )

### **5.) The next block creates the Shirley background and plots the initial guess.**
_Note that the plot intensities have been multiplied by 1e9!!_

In [ ]:
plot_columns = data[['Temperature',peak_input.region]].to_numpy()
plot_columns[:,1] = plot_columns[:,1]*1e9

# plot_columns = data[['Temperature',peak_input.region]]
temp_vals = data.columns.get_loc('Temperature')
activ_reg = peak_input.region
intensity_vals = data.columns.get_loc(peak_input.region)
print(str(peak_input.region) + " loaded!")
### Setting background
if peak_input.background_lower_bound is None or 0:
    peak_input.background_lower_bound = data['Temperature'][0]
if peak_input.background_upper_bound is None or 0:
    peak_input.background_upper_bound = data['Temperature'][-1]

ind_min = data['Temperature'].sub(peak_input.background_lower_bound).abs().lt(5).idxmax()
ind_max = data['Temperature'].sub(peak_input.background_upper_bound).abs().lt(5).idxmax()

x_min = plot_columns[ind_min,0]
x_max = plot_columns[ind_max,0]

y_min = plot_columns[ind_min,1]
y_max = plot_columns[ind_max,1]

print("Done!")

shirley_back_fin = shirley_func(ind_min, ind_max, plot_columns[:,0], plot_columns[:,1])

### Plotting initial parameters
### param = [peak binding energy, FWHM, area, % gauss/lorentz]

if peak_input.num_peaks is None:
    num_peaks = 2

if peak_input.param is None:
    param = [[74.6,1.0,100,80],[78,1.0,100,80]]

### if fitting couplets, only need half as many parameters as peaks

### Normalizing and zeroing the raw data for fitting
i = activ_reg

intensity_zeroed = np.zeros(plot_columns[:,0].shape)

for j in range(len(plot_columns)):
    intensity_zeroed[j] = plot_columns[j,1] - shirley_back_fin[j]

norm_factor = np.max(intensity_zeroed)

intensity_norm = intensity_zeroed / norm_factor    

### Plots the initial guess on the figure
initial_fit = []
for i in range(peak_input.num_peaks):
    initial_fit.append(sum_gl(plot_columns[ind_min:ind_max,0],*peak_input.param[i]) + shirley_back_fin[ind_min:ind_max])

initial_fit = np.array(initial_fit)

# make sure that you multiply the shirley_back_fin by (total number of initial fits minus 1!)
initial_fit_sum = 0
initial_fit_sum = np.sum(initial_fit,axis=0) - (peak_input.num_peaks-1) * shirley_back_fin[ind_min:ind_max]

fig = plt.figure(figsize=(10,5))

ax = fig.add_subplot(111)

ax.plot(plot_columns[ind_min:ind_max,0], plot_columns[ind_min:ind_max,1])       ## Plot Region in question
ax.plot(plot_columns[ind_min:ind_max,0],shirley_back_fin[ind_min:ind_max])    ## Plot Shirley background
ax.plot(plot_columns[ind_min:ind_max,0],initial_fit_sum)        ## Initial Fit
for i in range(len(initial_fit)):
    ax.plot(plot_columns[ind_min:ind_max,0],initial_fit[i])        ## Initial Fit

ax.set_title(activ_reg,fontweight='bold',fontsize=20)
ax.set_xlabel('Temperature [K]',fontweight='bold',fontsize=16)
ax.set_ylabel('Intensity',fontweight='bold',fontsize=16)
ax.tick_params(axis='both',which='major',labelsize=14)
fig.subplots_adjust(hspace = 0.4)

# plt.legend(['data', 'background', 'initial guess', 'final fit'])
plt.legend(['data', 'background', 'overall fit'])

ax.xaxis.set_minor_locator(MultipleLocator(100))

# ax[0].set_ylim(400,480)
# ax[1].set_ylim(0,500)

def mouse_move(event):
    x, y = event.xdata, event.ydata
    print(x, y)

plt.connect('motion_notify_event', mouse_move)

plt.show()

### **6.) Set your peak fitting min & max values**

In [ ]:
peak_param_id = ['E','F','area','m']

peak_model_list = []
peak_model_list.append(Model(sum_gl,prefix=peak_input.peak_id[0]))
pars = peak_model_list[0].make_params()

### Temperature
pars[peak_input.peak_id[0]+peak_param_id[0]].set(value=peak_input.param[0][0])

### FWHM (peak broadness, larger = broader)
pars[peak_input.peak_id[0]+peak_param_id[1]].set(value=peak_input.param[0][1],min=0,max=50)

### Area
pars[peak_input.peak_id[0]+peak_param_id[2]].set(value=peak_input.param[0][2],min=1e-11)

### Gaussian-Lorentzian weighting, higher = more Lorentzian
pars[peak_input.peak_id[0]+peak_param_id[3]].set(value=peak_input.param[0][3],min=0,max=100)

for i in range(1,peak_input.num_peaks):
    peak_model_list.append(Model(sum_gl,prefix=peak_input.peak_id[i]))
    pars.update(peak_model_list[i].make_params())

### Temperature
    pars[peak_input.peak_id[i]+peak_param_id[0]].set(value=peak_input.param[i][0])

### FWHM (peak broadness, larger = broader)
    pars[peak_input.peak_id[i]+peak_param_id[1]].set(value=peak_input.param[i][1],min=0,max=50)

### Area
    pars[peak_input.peak_id[i]+peak_param_id[2]].set(value=peak_input.param[i][2],min=1e-11)

### Gaussian-Lorentzian weighting, higher = more Lorentzian
    pars[peak_input.peak_id[i]+peak_param_id[3]].set(value=peak_input.param[i][3],min=0,max=100)
                
print(pars)
# print(peak_model_list)
peaks_all = np.array(peak_model_list)
model = np.sum(peaks_all,axis=0)


### **7.) Run this block to perform the fit.**
### Check the "\[\[Variables]]" section of the output for the fitted parameters

In [ ]:
fit = model.fit(intensity_zeroed[ind_min:ind_max],pars,x=plot_columns[ind_min:ind_max,0],method="leastsq")

print(fit.fit_report())

final = (fit.best_fit) + shirley_back_fin[ind_min:ind_max]
comps = fit.eval_components(x=plot_columns[ind_min:ind_max,0])

for i in peak_input.peak_id[:peak_input.num_peaks]:
    comps[i] = comps[i] + shirley_back_fin[ind_min:ind_max]

### **8.) Run this block to plot the fit. You may have to change the labels and such.**

In [ ]:
### Uncomment the next line to adjust the shift
peak_input.shift = 0

x_min_shift = x_min + peak_input.shift
x_max_shift = x_max + peak_input.shift

be_vals_shift = plot_columns[:,0] + peak_input.shift

fig = plt.figure(figsize=(7,5.5))

ax = fig.add_subplot(111)

line_wid = 2

ax.plot(be_vals_shift, plot_columns[:,1], color='grey')       ## Plot Region in question
ax.plot(be_vals_shift[ind_min:ind_max],shirley_back_fin[ind_min:ind_max])    ## Plot Shirley background

ax.plot(be_vals_shift[ind_min:ind_max], final, linewidth = line_wid)    ## Final Fit

for i in range(peak_input.num_peaks):
    ax.plot(be_vals_shift[ind_min:ind_max],comps[peak_input.peak_id[i]], label=peak_input.peak_id[i], linewidth = line_wid)

ax.set_title(activ_reg,fontweight='bold',fontsize=20)

ax.set_xlim(x_min_shift,x_max_shift)
# ax.set_ylim(20,35)

# ax.invert_xaxis()
ax.set_xlabel('Binding Energy [eV]',fontweight='bold',fontsize=16)
ax.set_ylabel('Intensity',fontweight='bold',fontsize=18)
ax.tick_params(axis='both',which='major',labelsize=18)
fig.subplots_adjust(hspace = 0.4)

plt.legend(['Data', 'Background', 'Fit'],fontsize=10,loc="upper right")

ax.xaxis.set_minor_locator(MultipleLocator(100))
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

def mouse_move(event):
    x, y = event.xdata, event.ydata
    print(x, y)

plt.connect('motion_notify_event', mouse_move)

plt.show()